# Using Callbacks

Callbacks are an integral part of Keras. Callbacks are used to get performance information, log progress, halt in the event of errors, tune parameters, save model state (in case of crash, etc.), finish training once loss is minimized. The list goes on.
Callbacks can be passed to fit, evaluate and predict methods of keras.Model. 

## Goals

The overarching goal is to learn to use callbacks for some typical tasks. These include:
- Reporting about training progress.
- Stoping once training no longer reduces loss.
- Tuning hyperparameters.
- Implementing adaptive learning rate decay.
- Finding an optimal batch-size for training.
- Putting some of this into ```my_keras_utils.py``` so that they can be easily called and reused.

## What's Here?

I continue working with MNIST data, which I began working with in [my first Keras models](first_model.ipynb). 

My **concrete objective** is to tune a model that does well on Kaggle: 97th percentile? That's tough, but I think I can make it work.

In [ ]:
import numpy as np
from datetime import datetime, time, timedelta

import pandas as pd
import tensorflow as tf
import kerastuner as kt
from tensorflow import keras
from tensorflow.keras import layers

import my_keras_utils as my_utils

In [ ]:
tf.__version__
tf.config.experimental.list_physical_devices('GPU')

In [ ]:
## Load our data.
## Since the load process is a little slow, the try-except allows us to re-run all 
## cells without having to wait. 
try:
    ## Raises NameError and loads data if X_train is not defined.
    X_train.shape
except NameError: 
    ((X_train, y_train), (X_dev, y_dev), (X_test, y_test)) = my_utils.load_kaggle_mnist()
X_train.shape




In [ ]:
## Let's use the dropout model from my first_model notebook.

inputs = keras.Input(shape=(784))
x = layers.experimental.preprocessing.Rescaling(1./255)(inputs)
x = layers.Dropout(rate = .05)(x)
x = layers.Dense(100, activation='relu',)(x)
x = layers.Dropout(rate = .15)(x)
x = layers.Dense(100, activation='relu',)(x)
x = layers.Dropout(rate = .15)(x)
outputs = layers.Dense(10, activation='softmax')(x)

model_dropout = keras.Model(inputs=inputs, outputs=outputs, name='Dropout')
model_dropout.summary()
optimizer = keras.optimizers.Adam(.001)

model_dropout.compile(optimizer=optimizer, 
                loss="sparse_categorical_crossentropy",
                metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")]
            )
init_weights = model_dropout.get_weights()


In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
                    monitor='loss', 
                    patience = 10, 
                    restore_best_weights = True)

adaptive_lr = keras.callbacks.ReduceLROnPlateau(
                    monitor='loss',
                    patience= 4,
                    cooldown= 8, 
                    factor=0.3,)
progress_update = my_utils.TimedProgressUpdate(1)
callbacks = [progress_update, early_stopping]
epochs = 0
initial_epoch = 0
batch_size = 128

In [ ]:
if True: ## reinitialize
    model_dropout.set_weights(init_weights)    
increment_epochs = 50
epochs += increment_epochs

history1 = model_dropout.fit(X_train, y_train, 
                        epochs=epochs,
                        initial_epoch = initial_epoch,
                        batch_size=batch_size, 
                        validation_data=(X_dev, y_dev),
                        callbacks = [progress_update],
                        verbose = 0)
initial_epoch += increment_epochs

In [ ]:
if True: ## reinitialize
    model_dropout.set_weights(init_weights)    
increment_epochs = 50
epochs += increment_epochs

history2 = model_dropout.fit(X_train, y_train, 
                        epochs=epochs,
                        initial_epoch = initial_epoch,
                        batch_size=batch_size, 
                        validation_data=(X_dev, y_dev),
                        callbacks = [adaptive_lr],
                        verbose =0)
initial_epoch += increment_epochs

In [ ]:
import matplotlib.pyplot as plt

def overlay_histories(histories, metric):
    fig, ax = plt.subplots()
    n = 0
    for h in histories:
        x = range(0,len(h.history[metric]))
        y = np.array(h.history[metric])
        label = 'history_' + str(n)
        ax.plot(x,y, label=label)
        n += 1
    ax.legend()

In [ ]:
overlay_histories([history1, history2], 'val_loss')

## Tuning

Time to work with Keras Tuner.

#### Read the Docs

Reading the documentation was really helpful. Note that the search function _will_ use callbacks. So, instead of worrying about 
the ```max_epochs```, you can (if you have the resources and time--which _is_ a resource) just add a realistic ```EarlyStopping``` callback. Below, I use this to stop if training loss is not decreasing.

In [ ]:
def model_builder(hp):
    ## Define the parameter search space.

    hp_dropout_x1 = hp.Float('rate1', min_value = .05, max_value = .5, step=.01)
    hp_dropout_w1 = hp.Float('rate2', min_value = .05, max_value = .5, step=.01)
    hp_dropout_w2 = hp.Float('rate3', min_value = .05, max_value = .5, step=.01)
    hp_l1_units = hp.Int('l1_units', min_value = 20, max_value = 200, step = 20)
    hp_l2_units = hp.Int('l2_units', min_value = 20, max_value = 200, step = 20)

    ## ### I need to learn about the options here. 'Choice' means "here are your choices"
    ## ### 'Int' is a different option that searches an integer range by steps.

    inputs = keras.Input(shape=(784))
    x = layers.experimental.preprocessing.Rescaling(1./255)(inputs)
    x = layers.Dropout(rate = hp_dropout_x1)(x)
    x = layers.Dense(hp_l1_units, activation='relu',)(x)
    x = layers.Dropout(rate = hp_dropout_w1)(x)
    x = layers.Dense(hp_l2_units, activation='relu',)(x)
    x = layers.Dropout(rate = hp_dropout_w2)(x)
    outputs = layers.Dense(10, activation='softmax')(x)
    model = keras.Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer = keras.optimizers.Adam(.003),
                    loss = "sparse_categorical_crossentropy",
                    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")]
    )

    return model

	


In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_loss', 
                     max_epochs = 120,
                     hyperband_iterations = 2,
                     factor = 3,
                     directory = 'ignored/kt_trials',
                     project_name = 'dropout_mnist',
                     )	

tuner.search_space_summary()

In [ ]:
## prevent bloated ipython output during training.
clear_output = my_utils.ClearTrainingOutput()
timed_update = my_utils.TimedProgressUpdate(3)
## stop when training loss is not happening.
train_loss_stopping = keras.callbacks.EarlyStopping(monitor='loss', 
                            patience = 10, 
                            restore_best_weights = False
                            )

adaptive_lr = keras.callbacks.ReduceLROnPlateau(
                    monitor='loss',
                    patience= 4,
                    cooldown= 8,
                    min_lr=.0003, 
                    factor=0.334,)

tuner_callbacks = [adaptive_lr, train_loss_stopping, timed_update]


In [6]:
start = datetime.now()
tuner.search(X_train, y_train, 
            epochs=120,
            batch_size=128, 
            validation_data=(X_dev, y_dev),
            callbacks = tuner_callbacks,
            verbose = 0
            )


Begin training of functional_1 at 00:37:21. Progress updates every 180.0 seconds.
Finished fitting at 00:37:27. Elapsed time 0:00:06.367594.


Begin training of functional_1 at 00:37:27. Progress updates every 180.0 seconds.
Finished fitting at 00:37:33. Elapsed time 0:00:05.999423.


Begin training of functional_1 at 00:37:34. Progress updates every 180.0 seconds.
Finished fitting at 00:37:40. Elapsed time 0:00:05.976725.


Begin training of functional_1 at 00:37:40. Progress updates every 180.0 seconds.
Finished fitting at 00:37:47. Elapsed time 0:00:06.236364.


Begin training of functional_1 at 00:37:47. Progress updates every 180.0 seconds.
Finished fitting at 00:37:53. Elapsed time 0:00:05.872332.


...

Begin training of functional_1 at 01:07:12. Progress updates every 180.0 seconds.
Finished fitting at 01:07:53. Elapsed time 0:00:40.987747.


Begin training of functional_1 at 01:07:54. Progress updates every 180.0 seconds.
Finished fitting at 01:08:51. Elapsed time 0:00:57.419433.


Begin training of functional_1 at 01:08:52. Progress updates every 180.0 seconds.
Finished fitting at 01:09:52. Elapsed time 0:00:59.932658.


Begin training of functional_1 at 01:09:52. Progress updates every 180.0 seconds.
Finished fitting at 01:10:38. Elapsed time 0:00:46.116739.


INFO:tensorflow:Oracle triggered exit


In [7]:
tuner.results_summary()

In [ ]:
end = datetime.now()
print(end - start)
tuner.results_summary()

## Ran the Hyperband, 

with the following params:
```
tuner = kt.Hyperband(model_builder,
                     objective = 'val_loss', 
                     max_epochs = 200,
                     hyperband_iterations = 10,
                     factor = 3,
                     directory = 'ignored/kt_trials',
                     project_name = 'dropout_mnist')	
###
tuner.search(X_train, y_train, 
            epochs=50,
            batch_size=128, 
            validation_data=(X_dev, y_dev),
            callbacks = tuner_callbacks,
            )
```
Which involved way too many iterations. I didn't notice the hyperband_iterations param (10!).

### os.path.normpath

Per [Issue #198](https://github.com/keras-team/keras-tuner/issues/198) you may need to add os.path.normpath() to the directory keyword arg in Windows and the path to the logs needs to be short. (E.g., you won't be able to use my_trials/this_type_of_trial/this_trial--too long. Try mt/t/this).

In [8]:
def rand_search_model_builder(hp):
    ## Define the hyperparameter search space.

    hp_dropout_x1 = hp.Float('rate1', min_value = .1, max_value = .4, step=.01)
    hp_dropout_w1 = hp.Float('rate2', min_value = .05, max_value = .4, step=.01)
    hp_dropout_w2 = hp.Float('rate3', min_value = .05, max_value = .4, step=.01)

    # Define the hypermodel
    inputs = keras.Input(shape=(784))
    x = layers.experimental.preprocessing.Rescaling(1./255)(inputs)
    x = layers.Dropout(rate = hp_dropout_x1)(x)
    x = layers.Dense(100, activation='relu',)(x)
    x = layers.Dropout(rate = hp_dropout_w1)(x)
    x = layers.Dense(100, activation='relu',)(x)
    x = layers.Dropout(rate = hp_dropout_w2)(x)
    outputs = layers.Dense(10, activation='softmax')(x)
    model = keras.Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer = keras.optimizers.Adam(.003),
                    loss = "sparse_categorical_crossentropy",
                    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")]
    )

    return model

#kt.tuners.RandomSearch?
new_tuner = kt.tuners.RandomSearch(rand_search_model_builder,
                     objective = 'val_loss', 
                     max_trials = 25,
                     executions_per_trial=1,
                     directory = os.path.normpath('ignored/mnist'),
                     project_name = 'rs')

new_tuner.search_space_summary()

adaptive_lr = keras.callbacks.ReduceLROnPlateau(
                    monitor='loss',
                    patience= 4,
                    cooldown= 8,
                    min_lr=.0003, 
                    factor=0.334,)

progress_update = my_utils.TimedProgressUpdate(2)

'''
new_tuner.search(X_train, y_train, 
            epochs=120,
            batch_size=128, 
            validation_data=(X_dev, y_dev),
            callbacks = [adaptive_lr, 
                        progress_update, 
                        #clear_output
                        ],
            verbose = 0
            )
'''

INFO:tensorflow:Reloading Oracle from existing project ignored\mnist\rs\oracle.json
INFO:tensorflow:Reloading Tuner from ignored\mnist\rs\tuner0.json


'\nnew_tuner.search(X_train, y_train, \n            epochs=120,\n            batch_size=128, \n            validation_data=(X_dev, y_dev),\n            callbacks = [adaptive_lr, \n                        progress_update, \n                        #clear_output\n                        ],\n            verbose = 0\n            )\n'

In [ ]:
def rand_search_model_builder_2(hp):
    ## Define the hyperparameter search space.

    hp_dropout_x1 = hp.Float('rate1', min_value = .27, max_value = .38, step=.001)
    hp_dropout_w1 = hp.Float('rate2', min_value = .05, max_value = .15, step=.001)
    hp_dropout_w2 = hp.Float('rate3', min_value = .11, max_value = .21, step=.001)

    # Define the hypermodel
    inputs = keras.Input(shape=(784))
    x = layers.experimental.preprocessing.Rescaling(1./255)(inputs)
    x = layers.Dropout(rate = hp_dropout_x1)(x)
    x = layers.Dense(100, activation='relu',)(x)
    x = layers.Dropout(rate = hp_dropout_w1)(x)
    x = layers.Dense(100, activation='relu',)(x)
    x = layers.Dropout(rate = hp_dropout_w2)(x)
    outputs = layers.Dense(10, activation='softmax')(x)
    model = keras.Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer = keras.optimizers.Adam(.003),
                    loss = "sparse_categorical_crossentropy",
                    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")]
    )

    return model

#kt.tuners.RandomSearch?
new_tuner = kt.tuners.RandomSearch(rand_search_model_builder_2,
                     objective = 'val_loss', 
                     max_trials = 25,
                     executions_per_trial=1,
                     directory = os.path.normpath('ignored/mnist'),
                     project_name = 'rs_2')

new_tuner.search_space_summary()

adaptive_lr = keras.callbacks.ReduceLROnPlateau(
                    monitor='loss',
                    patience= 4,
                    cooldown= 8,
                    min_lr=.0003, 
                    factor=0.334,)

progress_update = my_utils.TimedProgressUpdate(.5)

new_tuner.search(X_train, y_train, 
            epochs=1,
            batch_size=128, 
            validation_data=(X_dev, y_dev),
            callbacks = [adaptive_lr, 
                        progress_update, 
                        #clear_output
                        ],
            verbose = 0
            )

### A dubious fist run

I ran a random_search, lr = .001 and a 0-.5 range on the params; 50 epochs, 2 executions per trial. The results were okay, but it seemed like the number of epochs was too few. Also, the max_trials was too big. At least on the Surface, it took three minutes per trial. Ouch. 5 hours later...

### I ran a second time

This a second time. This time with more epochs (120), but 1 execution per trial and a slightly different band of paramters. There was a clear stand out

Trial ID: 39f9768e653c7179ddb0917bd84a4b9d
|-Score: 0.05155862867832184
|-Best step: 0
Hyperparameters:
|-rate1: 0.32999999999999985
|-rate2: 0.1
|-rate3: 0.16000000000000003

In [ ]:
new_tuner.results_summary()

## Baysian Tuning

Let's see how well the Baysian Optimizer does with the rand_search_builder.

In [11]:
bayesian_tuner = kt.tuners.BayesianOptimization(
            rand_search_model_builder,
            objective='val_loss',
            max_trials = 25,
            directory = os.path.normpath('ignored/mnist'),
            project_name = 'bayes'
)

bayesian_tuner.search(X_train, y_train, 
            epochs=120,
            batch_size=128, 
            validation_data=(X_dev, y_dev),
            callbacks = [adaptive_lr, 
                        progress_update, 
                        clear_output
                        ],
            verbose = 0
            )


INFO:tensorflow:Oracle triggered exit


In [12]:
bayesian_tuner.results_summary()